### 1. Packages


In [4]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-notebook') #additional modfication of plots using seaborng

### 2. Data loading 

In [5]:
#in case of unlabelled data we can label them
df = pd.read_csv("/Users/frantisek.grossmann/Desktop/Sets/crypto_data/LTC-USD.csv", names=['Time', 'Low', 'High', 'Open', 'Close', 'Volume'])
df.head() #all of the sets are indexed by time

,Time,Low,High,Open,Close,Volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


### 3. Joining BTC, LTC, BCH, ETH into one dataframe 

In [6]:
main_df = pd.DataFrame() # begin empty dataframe, in order to merge dataframes
pairs = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]
for pair in pairs:
    #print (ratios) for control 
    dataset=f"/Users/frantisek.grossmann/Desktop/Sets/crypto_data/{pair}.csv" #we created a dataset that f formats the strings using f

    df=pd.read_csv(dataset,names=['Time', 'Low', 'High', 'Open', 'Close', 'Volume'])
    #print(df.head()) for control purposes
    df.rename(columns={"Close": f"{pair}_Close","Volume": f"{pair}_Volume"}, inplace=True) #renaming of columns in the dataframe
    
    df.set_index("Time", inplace=True)
    
    df=df[[f"{pair}_Close",f"{pair}_Volume"]] # a dataframe of only Close and Volume
    
    #print(df.head()) for control 
    
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)
    
    #print(main_df.head()) for control 
    
#testing if the defined columns are available in the dataframe for each pair
for i in main_df.columns:
    print(i)

BTC-USD_Close
BTC-USD_Volume
LTC-USD_Close
LTC-USD_Volume
BCH-USD_Close
BCH-USD_Volume
ETH-USD_Close
ETH-USD_Volume


In [7]:
Seq_Length= 60 #the last 60 minutes of pricing data for prediction
Future_Time_Predict= 3 #how many minutes/hours into the future do we predict // functions like slicing
Predicted_Pair="LTC-USD"

# A simple function that runs through Current and Future columns and through an if function decides whether to buy or sell
def classify (current_price, future_price):
    if float(future_price) > float(current_price): #if future price in training data is greater than current return an int 1
        return 1 #buy
    else:
        return 0 #sell

In [9]:
main_df["Future"]=main_df[f"{Predicted_Pair}_Close"].shift(-Future_Time_Predict)
print(main_df[[f"{Predicted_Pair}_Close", "Future"]].head())
#prints out the current price in column LTC-USD_Close  96.580002
#prints out the 3 periods into the future price 96.500000

            LTC-USD_Close     Future
Time                                
1528968660      96.580002  96.500000
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997
1528968840      96.500000  96.440002
1528968900      96.389999  96.470001


In [18]:
main_df["Target"]=list(map(classify, main_df[f"{Predicted_Pair}_Close"],main_df["Future"]))
print(main_df[[f"{Predicted_Pair}_Close", "Future", "Target"]].head(20))

            LTC-USD_Close     Future  Target
Time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
1528969260      96.400002  96.449997       1
1528969320      96.400002  96.419998       1
1528969380      96.400002  96.400002       0
1528969440      96.449997  96.419998       0
1528969500      96.419998  96.570000       1
1528969560      96.400002  96.489998       1
1528969620      96.419998  96.589996       1
1528969680      96.570000  96.529999       0
1528969740      96.489998  96.629997       1
1528969800      96.589996  96.599998       1
